In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import scipy
import scipy.stats as stats
import os

In [2]:
# project files

from utils.variables import pack, unpack
from utils.display import table, tabulate
from utils.files import download_sheet, save, load
from utils.hash import digest

from fit_options import fit_options
from game_model import game
from rp_model import compute_rp, make_precomputed_columns
from initial_guess import make_initial_guess


In [3]:
# stuff for display

from IPython.lib.pretty import pretty, pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [4]:
# Load previous data

data = pd.read_pickle(fit_options.data_file)
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.000000,5753.0,5753.000000,5753.000000,5753.000000,5753.000000
mean,10.088128,587.835738,1.449852,587.856633,0.020895,3967.808100,3824.222892,12.129845,28.037546,37.195550,108.797845,1.011599,0.988589,0.994535,1.001912,1.005528,0.208206,31.919531,54.077699,147.916565,1.0,0.049539,1.963323,0.988258,1.027127
std,6.872198,372.904620,0.965124,372.913213,0.923897,947.858106,1011.929285,3.356745,2.614615,7.600589,15.702588,0.299950,0.059105,0.045200,0.118418,0.118363,0.044519,36.053210,26.406592,58.568292,0.0,0.427839,16.679667,0.036701,0.068786
min,1.000000,245.000000,1.000000,245.070000,-12.780000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.520000,0.890000,0.920000,0.800000,0.800000,0.130000,5.870000,25.000000,90.000000,1.0,0.000000,0.000000,0.790000,1.000000
25%,5.000000,352.000000,1.000000,351.640000,-0.280000,3300.000000,2992.680000,10.000000,26.000000,32.000000,98.000000,0.800000,1.000000,1.000000,1.000000,1.000000,0.180000,12.820000,35.000000,101.000000,1.0,0.000000,0.000000,1.000000,1.000000
50%,9.000000,449.000000,1.000000,449.090000,-0.020000,4000.000000,3815.820000,12.000000,28.000000,36.000000,103.000000,0.940000,1.000000,1.000000,1.000000,1.000000,0.200000,19.230000,44.000000,121.000000,1.0,0.000000,0.000000,1.000000,1.000000
75%,13.000000,695.000000,1.000000,695.210000,0.270000,4500.000000,4455.000000,14.000000,30.000000,41.000000,115.000000,1.200000,1.000000,1.000000,1.000000,1.000000,0.240000,43.980000,66.000000,202.000000,1.0,0.000000,0.000000,1.000000,1.000000
max,42.000000,3440.000000,6.000000,3440.330000,8.890000,6300.000000,6916.140000,23.000000,35.000000,74.000000,151.000000,2.410000,1.120000,1.080000,1.200000,1.200000,0.420000,414.030000,213.000000,511.000000,1.0,8.000000,342.000000,1.000000,1.440000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,530.69,-0.31,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.0,1.0,0.26,22.76,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.41,0.41,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.90,1.00,1.0,0.8,0.26,22.76,538.37,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.08,0.08,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.2,1.0,0.26,22.76,530.61,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arcanine,5.0,958.0,Calm,3.0,,957.85,-0.15,Calm,ArcanineCalm53,Main Skill Chance,Speed of Help,2500.0,2728.00,Fire,Skills,Extra Helpful S,16.0,27.0,31,Fiery Herb,130.0,1.31,1.11,1.00,1.0,1.2,0.14,84.52,937.81,ArcanineNeutral53,31.0,130.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arcanine,3.0,715.0,Lax,2.0,pokemon sleep general,714.65,-0.35,Lax,ArcanineLax32,Energy Recovery,Main Skill Chance,2500.0,2490.00,Fire,Skills,Extra Helpful S,16.0,27.0,29,Fiery Herb,130.0,1.44,1.00,1.08,1.0,0.8,0.14,61.23,750.25,ArcanineNeutral32,29.0,130.0,1,True,,,,0.0,0.0,1.0,1.0


In [5]:
# Load previous fit

x0, unpack_info = pack(make_initial_guess())

hash_value = digest(data, x0)
filename = fit_options.result_file(hash_value)

opt = load(filename)
sol = unpack(opt.x, unpack_info)

opt

    message: `ftol` termination condition is satisfied.
    success: True
     status: 2
          x: [ 2.562e-01  2.545e-01 ...  2.195e-01  9.957e-01]
       cost: 993.90653218049
       grad: [ 5.368e-06  2.602e-05 ... -1.415e-04  6.668e-06]
 optimality: 4.467174066696316
       nfev: 30
       njev: 23

In [6]:
# Helpers

def truncated_normal_sample(size, mu, sigma, lower, upper):
    return stats.truncnorm.rvs( (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma, size=size)

def round_noise(size):
    eps = 1e-6
    return truncated_normal_sample(size, mu=0.0, sigma=0.25, lower= -0.5+eps, upper= 0.5-eps)

def harmonic(a,b):
    return 2.0 / ( 1.0 / a + 1.0 / b )

In [7]:
# In the boostrap method,
# We create N synthetic datasets that are likely to represent the population
# Then we redo the fit on those, starting from the best fit in initial dataset.
# Finally, we collect statistics on the fitted variables between runs

# To create the synthetic datasets, we use sampling with replacement (allow duplicate)
# Because some Pokémon are rare, we use stratified sampling

stratas = data.groupby(['Pokemon'], group_keys=False)
target_group_size =  stratas.size().quantile(q=0.75)
    

n_boostraps = 50
all_opt_x = [opt.x]

In [8]:
cache_fit = True
boostrap_filename = f"./results/bootstrap-fit-{hash_value}.pickle"

if cache_fit and os.path.isfile(boostrap_filename):

    all_opt_x = load(boostrap_filename)
    print("Loaded from cache")

else:

    # WARNING: RUNNING THE OPTIMISATION N TIMES IS VERY LONG.
    #
    # YOU CAN INTERRUPT THE KERNEL (STOP BUTTON)
    # AND RUN THE NEXT CELLS IF YOU WANT TO SEE CURRENT PROGRESS
    #
    # THE COLLECTION `all_opt_x` IS DEFINED ABOVE THIS CELL
    # SO YOU CAN RESUME ADDING MORE RUNS

    while( len(all_opt_x) < n_boostraps ):

        print( "Boostrap run " + str(len(all_opt_x)) )

        # To create the synthetic datasets, we use sampling with replacement (allow duplicate)

        # We'll introduce a correction toward more equal sample size between Pokémon
        # We can motivate that by the fact stratified sampling should use population proportions
        # instead of current data proportions

        resampled = stratas.apply(lambda x: x.sample( round( harmonic(len(x),target_group_size) ), replace=True, ignore_index=True))

        # Add small noise to RP
        # We do so to simulate some unknown value that would round() to current RP

        referenceRP = resampled["RP"].to_numpy()
        referenceRP += round_noise(len(resampled))

        # Compute per sample information about help time, nature, sub-skills etc
        recomputed = make_precomputed_columns(resampled)

        # Put the pieces together

        def residual(x):
            return referenceRP - compute_rp(x, resampled, recomputed, unpack_info)

        # Redo the fit, starting from the optimal we found on current data
        opt2 = scipy.optimize.least_squares(residual, opt.x, **fit_options.least_squares_kwargs)

        # Collect results for stats
        all_opt_x.append(opt2.x)

    # Save
    save(boostrap_filename, all_opt_x)

Boostrap run 1
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.1579e+03                                    1.73e+06    
       1              2         1.1144e+03      4.35e+01       3.30e+00       2.19e+06    
       2              4         1.1067e+03      7.74e+00       1.19e+00       2.87e+05    
       3              5         1.1058e+03      8.37e-01       5.65e-01       1.50e+05    
       4              6         1.1051e+03      7.20e-01       3.46e-01       6.73e+04    
       5              7         1.1049e+03      2.40e-01       1.45e-01       1.81e+04    
       6              8         1.1049e+03      1.46e-02       2.69e-02       1.23e+02    
       7              9         1.1049e+03      5.99e-07       1.74e-04       2.98e-01    
`ftol` termination condition is satisfied.
Function evaluations 9, initial cost 1.1579e+03, final cost 1.1049e+03, first-order optimality 2.98e-01.
Boostrap run 2
   

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\least_squares.py:221: RuntimeWarning: overflow encountered in square
  z = (f / f_scale) ** 2


       1              6         1.0877e+03      5.91e+01       1.32e+00       2.67e+06    
       2              8         1.0860e+03      1.68e+00       6.59e-01       4.95e+06    
       3              9         1.0813e+03      4.72e+00       1.65e-01       7.73e+05    
       4             10         1.0807e+03      5.89e-01       3.30e-01       1.35e+05    
       5             11         1.0805e+03      2.07e-01       6.59e-01       5.90e+03    
       6             14         1.0805e+03      6.32e-03       8.24e-02       2.88e+02    
       7             16         1.0805e+03      6.92e-04       4.12e-02       8.43e+00    
       8             20         1.0805e+03      3.90e-06       1.29e-03       3.43e+00    
`ftol` termination condition is satisfied.
Function evaluations 20, initial cost 1.1468e+03, final cost 1.0805e+03, first-order optimality 3.43e+00.
Boostrap run 6
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0         

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\least_squares.py:221: RuntimeWarning: overflow encountered in square
  z = (f / f_scale) ** 2


       1              6         1.0680e+03      6.04e+01       1.32e+00       1.01e+07    
       2              9         1.0539e+03      1.41e+01       8.24e-02       7.60e+06    
       3             13         1.0454e+03      8.50e+00       1.29e-03       9.08e+06    
       4             14         1.0353e+03      1.01e+01       1.29e-03       2.27e+06    
       5             15         1.0335e+03      1.87e+00       1.29e-03       1.51e+04    
       6             16         1.0330e+03      4.54e-01       2.57e-03       2.57e+04    
       7             17         1.0325e+03      5.08e-01       5.15e-03       2.10e+03    
       8             18         1.0317e+03      7.90e-01       1.03e-02       4.02e+03    
       9             19         1.0309e+03      8.25e-01       2.06e-02       4.86e+03    
      10             20         1.0305e+03      4.33e-01       4.12e-02       1.35e+04    
      11             21         1.0302e+03      2.55e-01       8.24e-02       5.17e+03    

In [9]:
# Stats
# We'll use the median as estimate for the mean.
# And 1.4826 * < median absolute deviation from the median > as estimate for std

center = np.median(all_opt_x, axis=0)

interval_std = 1.96 * np.std( all_opt_x, axis=0) # 95% confidence region
interval_mad = 3 * np.median( list(map(lambda x: np.abs(center-x), all_opt_x)), axis=0) # 95% confidence region

sol_center = unpack(center, unpack_info)
sol_interval_std = unpack(interval_std, unpack_info)
sol_interval_mad = unpack(interval_mad, unpack_info)

# Pretty display
pd.DataFrame({
    
    "Pokemon":game.pokedex.data["Pokemon"], 
    
    "ing%": sol_center["Pokemons ing fractions"]*100.0,
    #"conf (ing)": sol_interval_std["Pokemons ing fractions"]*100.0,
    "conf (ing)*": sol_interval_mad["Pokemons ing fractions"]*100.0,

    "skill% * skillValue": sol_center["Pokemons skill products"], 
    #"conf (skill)": sol_interval_std["Pokemons skill products"],
    "conf (skill)*": sol_interval_mad["Pokemons skill products"],


}).set_index("Pokemon")

,ing%,conf (ing)*,skill% * skillValue,conf (skill)*
Pokemon,,,,
Bulbasaur,25.626673,0.073799,16.729813,0.098730
Ivysaur,25.450137,0.044855,16.705872,0.030274
Venusaur,26.564516,0.055975,18.438908,0.050956
Charmander,20.036129,0.051229,9.683627,0.084200
Charmeleon,22.658198,0.047610,14.062530,0.047857
Charizard,22.379606,0.028799,14.039124,0.052125
Squirtle,26.994385,0.047731,17.634126,0.082486
Wartortle,27.047718,0.037712,17.579687,0.027526
Blastoise,27.428278,0.056942,18.433115,0.022404


In [10]:
# Explain what we did with the re-sampling target size
# pd.DataFrame({'before': stratas.size(), 'after': stratas.apply(lambda x: round( harmonic(len(x), target_group_size) )) })

In [11]:
data[data["Pokemon"]=="Slaking"]

,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
2077,Slaking,27.0,2648.0,Calm,6.0,,2645.62,-2.38,Calm,SlakingCalm276Skill Level Up MEnergy Recovery ...,Main Skill Chance,Speed of Help,3800.0,3962.64,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.90,1.10,1.00,1.0,1.2,0.34,304.43,2545.01,SlakingNeutral276Skill Level Up MEnergy Recove...,108.0,110.0,1,True,Skill Level Up M,Energy Recovery Bonus,,0.0,0.0,1.0,1.22
2078,Slaking,27.0,1615.0,Gentle,3.0,,1616.87,1.87,Gentle,SlakingGentle273Berry Finding SHelping bonus,Main Skill Chance,Energy Recovery,3800.0,3602.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.99,1.00,0.92,1.0,1.2,0.34,115.61,1616.08,SlakingNeutral273Berry Finding SHelping bonus,162.0,110.0,1,True,Berry Finding S,Helping bonus,,0.0,0.0,1.0,1.22
2079,Slaking,27.0,2637.0,Quiet,6.0,,2636.77,-0.23,Quiet,SlakingQuiet276Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3602.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.99,1.00,1.00,1.2,1.0,0.34,414.03,2628.21,SlakingNeutral276Skill Level Up MSkill Trigger M,108.0,110.0,1,True,Skill Level Up M,Skill Trigger M,,0.0,0.0,1.0,1.00
2080,Slaking,29.0,2681.0,Quiet,6.0,,2681.36,0.36,Quiet,SlakingQuiet296Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3587.20,Normal,Berries,Ingredient Magnet S,12.0,28.0,56,Snoozy Tomato,110.0,1.00,1.00,1.00,1.2,1.0,0.34,414.03,2672.46,SlakingNeutral296Skill Level Up MSkill Trigger M,112.0,110.0,1,True,Skill Level Up M,Skill Trigger M,,0.0,0.0,1.0,1.00
2081,Slaking,30.0,2831.0,Quiet,6.0,,2831.79,0.79,Quiet,SlakingQuiet306Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3579.60,Normal,Berries,Ingredient Magnet S,12.0,28.0,57,Snoozy Tomato,110.0,1.00,1.00,1.00,1.2,1.0,0.34,414.03,2799.06,SlakingNeutral306Skill Level Up MSkill Trigger M,114.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.0,1.00
2082,Slaking,32.0,2859.0,Quiet,6.0,,2859.48,0.48,Quiet,SlakingQuiet326Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3564.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,60,Snoozy Tomato,110.0,1.00,1.00,1.00,1.2,1.0,0.34,414.03,2828.99,SlakingNeutral326Skill Level Up MSkill Trigger M,120.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.0,1.00
2083,Slaking,33.0,2905.0,Quiet,6.0,,2905.24,0.24,Quiet,SlakingQuiet336Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3556.80,Normal,Berries,Ingredient Magnet S,12.0,28.0,62,Snoozy Tomato,110.0,1.01,1.00,1.00,1.2,1.0,0.34,414.03,2876.02,SlakingNeutral336Skill Level Up MSkill Trigger M,124.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.0,1.00
2892,Slaking,22.0,965.0,Serious,1.0,https://discord.com/channels/11339232641467064...,964.57,-0.43,Neutral,SlakingNeutral221Inventory Up L,-,-,3800.0,3640.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,49,Snoozy Tomato,110.0,0.98,1.00,1.00,1.0,1.0,0.34,58.90,965.00,SlakingNeutral221Inventory Up L,98.0,110.0,1,True,Inventory Up L,,,0.0,0.0,1.0,1.18
2893,Slaking,26.0,1236.0,Timid,4.0,https://discord.com/channels/11339232641467064...,1236.23,0.23,Timid,SlakingTimid264Skill Level Up MSkill Level Up S,EXP Gains,Speed of Help,3800.0,3971.00,Normal,Berries,Ingredient Magnet S,12.0,28.0,53,Snoozy Tomato,110.0,0.90,1.10,1.00,1.0,1.0,0.34,159.65,1359.60,SlakingNeutral264Skill Level Up MSkill Level Up S,106.0,110.0,1,True,Skill Level Up M,Skill Level Up S,,0.0,0.0,1.0,1.00
2894,Slaking,27.0,1537.0,Naughty,3.0,https://discord.com/channels/11339232641467064...,1537.50,0.50,Naughty,SlakingNaughty273Berry Finding SInventory Up M,Speed of Help,Main Skill Chance,3800.0,3

In [12]:
data[data["Pokemon"]=="Glaceon"]


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
1153,Glaceon,9.0,773.0,Lonely,2.0,pokemon sleep general,773.23,0.23,Lonely,GlaceonLonely92,Speed of Help,Energy Recovery,3200.0,2833.92,Ice,Skills,Cooking Power-Up S,12.0,32.0,40,Moomoo Milk,98.0,1.27,0.9,0.92,1.0,1.0,0.22,78.63,754.21,GlaceonNeutral92,40.0,98.0,1,True,,,,0.0,0.0,1.0,1.00
1154,Glaceon,9.0,815.0,Relaxed,2.0,,814.79,-0.21,Relaxed,GlaceonRelaxed92,Energy Recovery,EXP Gains,3200.0,3148.80,Ice,Skills,Cooking Power-Up S,12.0,32.0,40,Moomoo Milk,98.0,1.14,1.0,1.08,1.0,1.0,0.22,78.63,754.63,GlaceonNeutral92,40.0,98.0,1,True,,,,0.0,0.0,1.0,1.00
1163,Glaceon,14.0,891.0,Bashful,2.0,,891.00,0.00,Neutral,GlaceonNeutral142Inventory Up M,-,-,3200.0,3116.80,Ice,Skills,Cooking Power-Up S,12.0,32.0,45,Moomoo Milk,98.0,1.15,1.0,1.00,1.0,1.0,0.22,78.63,891.00,GlaceonNeutral142Inventory Up M,45.0,98.0,1,True,Inventory Up M,,,0.0,0.0,1.0,1.13
2889,Glaceon,10.0,1230.0,Gentle,4.0,https://discord.com/channels/11339232641467064...,1230.04,0.04,Gentle,GlaceonGentle104Skill Level Up M,Main Skill Chance,Energy Recovery,3200.0,3142.40,Ice,Skills,Cooking Power-Up S,12.0,32.0,41,Moomoo Milk,98.0,1.14,1.0,0.92,1.0,1.2,0.22,149.93,1166.04,GlaceonNeutral104Skill Level Up M,41.0,98.0,1,True,Skill Level Up M,,,0.0,0.0,1.0,1.00
2890,Glaceon,11.0,1235.0,Gentle,4.0,https://discord.com/channels/11339232641467064...,1234.93,-0.07,Gentle,GlaceonGentle114Skill Level Up M,Main Skill Chance,Energy Recovery,3200.0,3136.00,Ice,Skills,Cooking Power-Up S,12.0,32.0,42,Moomoo Milk,98.0,1.14,1.0,0.92,1.0,1.2,0.22,149.93,1171.46,GlaceonNeutral114Skill Level Up M,42.0,98.0,1,True,Skill Level Up M,,,0.0,0.0,1.0,1.00
2891,Glaceon,12.0,1251.0,Gentle,4.0,https://discord.com/channels/11339232641467064...,1250.73,-0.27,Gentle,GlaceonGentle124Skill Level Up M,Main Skill Chance,Energy Recovery,3200.0,3129.60,Ice,Skills,Cooking Power-Up S,12.0,32.0,43,Moomoo Milk,98.0,1.15,1.0,0.92,1.0,1.2,0.22,149.93,1187.33,GlaceonNeutral124Skill Level Up M,43.0,98.0,1,True,Skill Level Up M,,,0.0,0.0,1.0,1.00
